# Import libraries

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow import keras
from tensorflow.python.platform import gfile
from PIL import Image
from tensorflow.keras.applications.resnet_v2 import preprocess_input

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Sequential,Model
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
from glob import glob

# Getting the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/chest_xray.zip','r')as zip_ref:
  zip_ref.extractall('')

In [ ]:
import os
os.listdir()

#Checking out the images and the data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
IMAGE_SIZE = [224, 224]
#test_path = 'chest_xray/test'
train_path = 'chest_xray/train'
valid_path = 'chest_xray/test'

In [ ]:
res = ResNet50V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:

for layer in res.layers[-10:]:
  layer.trainable = False

In [ ]:
folders = glob('/content/chest_xray/train/*')

In [ ]:
x = Flatten()(res.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=res.input, outputs=prediction)

In [ ]:
model.summary()

# Training the model

In [ ]:
batch_size=16

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                   zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
%%time
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=4,
  steps_per_epoch=20,
  validation_steps=len(test_set)
)

In [ ]:
result  = model.evaluate_generator(test_set, steps=len(test_set), verbose=1)
print('Loss:', result[0])
print('Accuracy:', result[1])

# Evaluation of the model

**Model saving and loading**

In [ ]:
model.save('/content/Pneumonia-Detection.h5')

In [ ]:
new_model = keras.models.load_model('Pneumonia-Detection.h5')